In [11]:
import pandas as pd
from spacy.cli import download
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter
import spacy

In [12]:
train_data = pd.read_csv('augmented_data_with_gpt2.csv')
val_data = pd.read_csv('preprocessed_val_data.csv')

In [13]:
train_data.head()

,comments_url,id,title,body,issue_url,pr_url,labels,pr_number,filename,status,additions,deletions,changes,all_comments,all_text,is_augmented
0,https://api.github.com/repos/kubernetes/kubern...,463714000,etcd image should be world executable to allow...,Part of https://github.com/kubernetes/kubeadm/...,https://github.com/kubernetes/kubernetes/issue...,https://github.com/kubernetes/kubernetes/pull/...,['area/security'],79722.0,['cluster/images/etcd/Makefile'],['modified'],[26],[15],[41],/assign randomvariable\r\n/lifecycle active /p...,etcd image world executable allow run non root...,False
1,https://api.github.com/repos/kubernetes/kubern...,388336320,"When using IPVS with graceful termination, con...",**What happened**:\r\nAfter updating the pods ...,https://github.com/kubernetes/kubernetes/issue...,https://github.com/kubernetes/kubernetes/pull/...,['area/ipvs'],71834.0,['pkg/proxy/ipvs/proxier.go'],['modified'],[16],[0],[16],NaN,ipvs graceful termination connection service s...,False
2,https://api.github.com/repos/kubernetes/kubern...,371198878,enforce-node-allocatable=kube-reserved/system-...,**Is this a BUG REPORT or FEATURE REQUEST?**:\...,https://github.com/kubernetes/kubernetes/issue...,https://github.com/kubernetes/kubernetes/pull/...,['area/kubelet'],69960.0,"['pkg/kubelet/cm/BUILD', 'pkg/kubelet/cm/conta...","['modified', 'modified', 'added', 'renamed', '...","[3, 128, 46, 0, 0]","[2, 6, 0, 26, 0]","[5, 134, 46, 26, 0]",@PatrickLang: You must be a member of the [kub...,enforce node allocatable kube reserve system r...,False
3,https://api.github.com/repos/kubernetes/kubern...,505792177,Pods error out with 'Topology Affinity Error'...,**What happened**:\r\n\r\nWith the latest fix ...,https://github.com/kubernetes/kubernetes/issue...,https://github.com/kubernetes/kubernetes/pull/...,['area/kubelet'],83777.0,['pkg/kubelet/cm/topologymanager/policy_single...,"['modified', 'modified', 'modified']","[1, 0, 18]","[1, 10, 0]","[2, 10, 18]",@kubernetes/sig-node-bugs @Deepthidharwar: Rei...,pod error topology affinity error resource lim...,False
4,https://api.github.com/repos/kubernetes/kubern...,445262273,LoadBalancer Service stuck on pending when Ena...,<!-- Please use this template while reporting ...,https://github.com/kubernetes/kubernetes/issue...,https://github.com/kubernetes/kubernetes/pull/...,['area/cloudprovider'],78012.0,['staging/src/k8s.io/legacy-cloud-providers/az...,"['modified', 'modified', 'modified', 'modified...","[2, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[2, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[4, 2, 6, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...",/milestone v1.15 ref: #78012,loadbalancer service stick pende enabletcprese...,False


In [14]:
# check for null values
train_data.isnull().sum()


comments_url    2581
id                 0
title              0
body               3
issue_url       2581
pr_url          2581
labels             0
pr_number       2581
filename        2581
status          2581
additions       2581
deletions       2581
changes         2581
all_comments    2622
all_text           0
is_augmented       0
dtype: int64

In [15]:
val_data.head()

,comments_url,id,title,body,issue_url,pr_url,labels,pr_number,filename,status,additions,deletions,changes,all_comments,all_text
0,https://api.github.com/repos/kubernetes/kubern...,826157852,newTestResourceQuota causes clusters to fail c...,#### What happened:\r\nI ran conformance tests...,https://github.com/kubernetes/kubernetes/issue...,https://github.com/kubernetes/kubernetes/pull/...,['area/test'],100056,['test/e2e/apimachinery/resource_quota.go'],['modified'],[2],[4],[6],/assign\r\n my guess is the CA configmap creat...,newtestresourcequota cause cluster fail confor...
1,https://api.github.com/repos/kubernetes/kubern...,714272397,Failed nodes checks when their Azure VMSS was ...,**What happened**:\r\n\r\nWe saw several occur...,https://github.com/kubernetes/kubernetes/issue...,https://github.com/kubernetes/kubernetes/pull/...,"['area/cloudprovider', 'area/provider/azure']",95289,['staging/src/k8s.io/legacy-cloud-providers/az...,"['modified', 'modified', 'modified']","[54, 3, 9]","[0, 0, 0]","[54, 3, 9]",@bpineau: This issue is currently awaiting tri...,fail node check azure vmss delete happen see o...
2,https://api.github.com/repos/kubernetes/kubern...,2157915212,Docs: When the `--bind-address` parameter of k...,### What happened?\n\nThe `--bind-address` par...,https://github.com/kubernetes/kubernetes/issue...,https://github.com/kubernetes/kubernetes/pull/...,['area/kube-proxy'],123545,['cmd/kube-proxy/app/server.go'],['modified'],[2],[2],[4],/sig network\r\n /assign Thank you for this. \...,docs address parameter kube proxy configure ip...
3,https://api.github.com/repos/kubernetes/kubern...,569793826,net.Listen listens on all available unicast an...,<!-- Please use this template while reporting ...,https://github.com/kubernetes/kubernetes/issue...,https://github.com/kubernetes/kubernetes/pull/...,"['area/kubelet', 'area/apiserver']",97245,"['cmd/kube-apiserver/app/options/options.go', ...","['modified', 'modified', 'modified', 'modified']","[1, 1, 2, 2]","[1, 1, 2, 2]","[2, 2, 4, 4]",/kind cleanup\r\n/help wanted\r\n/good-first-i...,listen available unicast anycast ip address ip...
4,https://api.github.com/repos/kubernetes/kubern...,560555732,kube-proxy can panic if len(HealthzBindAddress...,<!-- Please use this template while reporting ...,https://github.com/kubernetes/kubernetes/issue...,https://github.com/kubernetes/kubernetes/pull/...,['area/ipvs'],87870,"['cmd/kube-proxy/app/server.go', 'cmd/kube-pro...","['modified', 'modified', 'modified', 'modified']","[1, 1, 1, 13]","[1, 1, 1, 10]","[2, 2, 2, 23]",```\r\n hs.lastQueued.Store(hs.clock.Now()) \r...,kube proxy panic use template reporting bug pr...


In [16]:
val_data.isnull().sum()

comments_url    0
id              0
title           0
body            0
issue_url       0
pr_url          0
labels          0
pr_number       0
filename        0
status          0
additions       0
deletions       0
changes         0
all_comments    4
all_text        0
dtype: int64

In [17]:
# remove all_text that have null values
train_data = train_data[train_data['all_text'].notnull()]
val_data = val_data[val_data['all_text'].notnull()]

# remove all_text that has value 'NaN ' or 'nan'
train_data = train_data[~train_data['all_text'].isin(['NaN ', 'nan'])]
val_data = val_data[~val_data['all_text'].isin(['NaN ', 'nan'])]

Check the label distribution

In [18]:
# Flatten the list of labels if it's a list of lists
flattened_labels = []
for label_list in train_data["labels"]:
    # Convert string representation of list to actual list
    if isinstance(label_list, str):
        # Remove brackets and split by comma
        clean_label_list = label_list.strip('[]').replace("'", "").split(', ')
    else:
        clean_label_list = label_list
    
    # Add each label to the flattened list
    flattened_labels.extend(clean_label_list)

# Print labels distribution count
label_counts = Counter(flattened_labels)
print("Label Distribution Count (Training Data):")

for i, (label, count) in enumerate(label_counts.most_common()):
    print(f"{i+1}. {label}: {count}")

Label Distribution Count (Training Data):
1. area/kubelet: 264
2. area/test: 221
3. area/security: 198
4. area/ipvs: 198
5. area/cloudprovider: 198
6. area/apiserver: 198
7. area/code-generation: 198
8. area/kubeadm: 198
9. area/release-eng: 198
10. area/conformance: 198
11. area/provider/gcp: 198
12. area/provider/azure: 198
13. area/kubectl: 198
14. area/dependency: 198
15. area/custom-resources: 198
16. area/kube-proxy: 198
17. area/e2e-test-framework: 198
18. area/provider/openstack: 198


In [19]:
# Flatten the list of labels if it's a list of lists
flattened_labels = []
for label_list in val_data["labels"]:
    # Convert string representation of list to actual list
    if isinstance(label_list, str):
        # Remove brackets and split by comma
        clean_label_list = label_list.strip('[]').replace("'", "").split(', ')
    else:
        clean_label_list = label_list
    
    # Add each label to the flattened list
    flattened_labels.extend(clean_label_list)

# Print labels distribution count
label_counts = Counter(flattened_labels)
print("Label Distribution Count (Validation Data):")

for i, (label, count) in enumerate(label_counts.most_common()):
    print(f"{i+1}. {label}: {count}")

Label Distribution Count (Validation Data):
1. area/kubelet: 70
2. area/test: 60
3. area/cloudprovider: 30
4. area/kubectl: 28
5. area/apiserver: 27
6. area/provider/azure: 16
7. area/kubeadm: 6
8. area/kube-proxy: 5
9. area/ipvs: 5
10. area/code-generation: 4
11. area/custom-resources: 4
12. area/dependency: 4
13. area/e2e-test-framework: 3
14. area/conformance: 3
15. area/provider/gcp: 3
16. area/security: 2
17. area/release-eng: 2
18. area/provider/openstack: 1


Stopwords Removal

In [20]:
# remove stopwords using spaCy
try:
    nlp = spacy.load('en_core_web_sm')
except OSError:
    download('en_core_web_sm')
    nlp = spacy.load('en_core_web_sm')

train_data['all_text'] = train_data['all_text'].apply(lambda x: ' '.join([word.text for word in nlp(x) if not word.is_stop]))
val_data['all_text'] = val_data['all_text'].apply(lambda x: ' '.join([word.text for word in nlp(x) if not word.is_stop]))

In [21]:
# check null values in all_text column
# Display null values for all columns
print(f"Null all_text column : {train_data['all_text'].isnull().sum()}")

Null all_text column : 0


Data Lemmatization

In [22]:
# lematize the text using spaCy
train_data['all_text'] = train_data['all_text'].apply(lambda x: ' '.join([word.lemma_ for word in nlp(x)]))
val_data['all_text'] = val_data['all_text'].apply(lambda x: ' '.join([word.lemma_ for word in nlp(x)]))

In [23]:
# check all_text column for null values
print(f"Null all_text column : {train_data['all_text'].isnull().sum()}")

Null all_text column : 0


# Export the Data

In [24]:
train_data.head()

,comments_url,id,title,body,issue_url,pr_url,labels,pr_number,filename,status,additions,deletions,changes,all_comments,all_text,is_augmented
0,https://api.github.com/repos/kubernetes/kubern...,463714000,etcd image should be world executable to allow...,Part of https://github.com/kubernetes/kubeadm/...,https://github.com/kubernetes/kubernetes/issue...,https://github.com/kubernetes/kubernetes/pull/...,['area/security'],79722.0,['cluster/images/etcd/Makefile'],['modified'],[26],[15],[41],/assign randomvariable\r\n/lifecycle active /p...,etcd image world executable allow run non root...,False
1,https://api.github.com/repos/kubernetes/kubern...,388336320,"When using IPVS with graceful termination, con...",**What happened**:\r\nAfter updating the pods ...,https://github.com/kubernetes/kubernetes/issue...,https://github.com/kubernetes/kubernetes/pull/...,['area/ipvs'],71834.0,['pkg/proxy/ipvs/proxier.go'],['modified'],[16],[0],[16],NaN,ipvs graceful termination connection service s...,False
2,https://api.github.com/repos/kubernetes/kubern...,371198878,enforce-node-allocatable=kube-reserved/system-...,**Is this a BUG REPORT or FEATURE REQUEST?**:\...,https://github.com/kubernetes/kubernetes/issue...,https://github.com/kubernetes/kubernetes/pull/...,['area/kubelet'],69960.0,"['pkg/kubelet/cm/BUILD', 'pkg/kubelet/cm/conta...","['modified', 'modified', 'added', 'renamed', '...","[3, 128, 46, 0, 0]","[2, 6, 0, 26, 0]","[5, 134, 46, 26, 0]",@PatrickLang: You must be a member of the [kub...,enforce node allocatable kube reserve system r...,False
3,https://api.github.com/repos/kubernetes/kubern...,505792177,Pods error out with 'Topology Affinity Error'...,**What happened**:\r\n\r\nWith the latest fix ...,https://github.com/kubernetes/kubernetes/issue...,https://github.com/kubernetes/kubernetes/pull/...,['area/kubelet'],83777.0,['pkg/kubelet/cm/topologymanager/policy_single...,"['modified', 'modified', 'modified']","[1, 0, 18]","[1, 10, 0]","[2, 10, 18]",@kubernetes/sig-node-bugs @Deepthidharwar: Rei...,pod error topology affinity error resource lim...,False
4,https://api.github.com/repos/kubernetes/kubern...,445262273,LoadBalancer Service stuck on pending when Ena...,<!-- Please use this template while reporting ...,https://github.com/kubernetes/kubernetes/issue...,https://github.com/kubernetes/kubernetes/pull/...,['area/cloudprovider'],78012.0,['staging/src/k8s.io/legacy-cloud-providers/az...,"['modified', 'modified', 'modified', 'modified...","[2, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[2, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[4, 2, 6, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...",/milestone v1.15 ref: #78012,loadbalancer service stick pende enabletcprese...,False


In [25]:
# count nan values in the train_data
train_data.isna().sum()

comments_url    2581
id                 0
title              0
body               3
issue_url       2581
pr_url          2581
labels             0
pr_number       2581
filename        2581
status          2581
additions       2581
deletions       2581
changes         2581
all_comments    2622
all_text           0
is_augmented       0
dtype: int64

In [26]:
# export the train_data and val_data to csv
train_data.to_csv('preprocessed_augmented_train_data.csv', index=False)
val_data.to_csv('preprocessed_val_data.csv', index=False)